## First SQL Project

This is my very first project on working with SQL, It is a guided project on the Data Analyst path on [DataQuest](https://app.dataquest.io/)

In [4]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [5]:
%%sql
SELECT *
  FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


``
Querying the fact table in the database.
``

In [9]:
%%sql
SELECT *
    FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The table is an information about several countries. In the next couple of cells, I will be performing some summary statistics using SQL. 

In [14]:
%%sql
SELECT MIN(population) AS min_population,
       MAX(population) AS max_population,
       MIN(population_growth) AS min_population_growth,
       MAX(population_growth) AS max_population_growth
FROM facts;

 * sqlite:///factbook.db
Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


Interestingly, there is(are) country(ies) with zero population and zero growth. I will investigate this further.

In [16]:
%%sql
SELECT name
FROM facts
WHERE population == 0;

 * sqlite:///factbook.db
Done.


name
Antarctica


Okay, that's is understandable..I would not want to live in Antarctica as well. But what about the zero growth rate?

In [17]:
%%sql
SELECT name
FROM facts
WHERE population_growth == 0;

 * sqlite:///factbook.db
Done.


name
Holy See (Vatican City)
Cocos (Keeling) Islands
Greenland
Pitcairn Islands


Cool, there are four of such countries. And surprisingly, Antarctica is not among. Let's investigate the growth rate at Antarctica maybe there is any hope of me joining the settlement.. 

In [20]:
%%sql
SELECT *
FROM facts
WHERE name == 'Antarctica';

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


Oh, that's missing! And there goes off my hope of leaving sunny Africa for ice cold Antarctica. I will like to turn my attention backward to the country with maximum population, at least so that I can know where not to pay a visit!

In [24]:
%%sql
SELECT name
FROM facts
WHERE population == (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


name
World


**Oh, interesting...It is the whole world! Time for me to join the Elon's SpaceX wagon..**. Since I now know that the world is part of the rows, then I will exclude it when calculating the summary statistics. 

In [33]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


`
There is still a country with a population over a billion, probably China! Let's confirm..
`

In [26]:
%%sql
SELECT name
    FROM facts
WHERE population == 1367485388;

 * sqlite:///factbook.db
Done.


name
China


Said it..Now, I will compute the average population the world over and the average area. I will of course be excluding the "world" row in the data. 

In [32]:
%%sql
SELECT AVG(population) AS avg_pop,
       AVG(area) AS avg_area
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


To conclude the database exploration project, I will be querying the database for countries for that are densely populated (to cross them out of my traveling list) but looking out for population over the average and the area below the average.

In [36]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population)
                           FROM facts
                    WHERE name != 'World') 
AND 
     area < (SELECT AVG(area)
                    FROM facts
             WHERE name != 'World');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


There they are! No go area..

Thank you for sticking with me, I hope you learned something...you can reach me on ogunfoworalawal@gmail.com